# Mask R-CNN - Train on NewShapes Dataset

### Notes from implementation

This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string
import mrcnn.visualize as visualize
import mrcnn.new_shapes as new_shapes
from mrcnn.prep_notebook import prep_newshapes_train2
import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)


##------------------------------------------------------------------------------------
## Build configuration object 
##------------------------------------------------------------------------------------
config                    = new_shapes.NewShapesConfig()
config.FCN_LAYERS         = False
config.BATCH_SIZE         = 2                                 # Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU     = config.BATCH_SIZE                 # Must match BATCH_SIZE
config.STEPS_PER_EPOCH    = 2
config.LEARNING_RATE      = 0.000001
                          
config.EPOCHS_TO_RUN      = 300
config.FCN_INPUT_SHAPE    = config.IMAGE_SHAPE[0:2]
config.LAST_EPOCH_RAN     = 0
config.WEIGHT_DECAY       = 2.0e-4
config.VALIDATION_STEPS   = 100
config.REDUCE_LR_FACTOR   = 0.2
config.REDUCE_LR_COOLDOWN = 30
config.REDUCE_LR_PATIENCE = 40
config.MIN_LR             = 1.0e-10
config.TRAINING_IMAGES    = 10000
config.VALIDATION_IMAGES  = 2500
config.CHECKPOINT_FOLDER  = 'newshape_fcn' 

model_file  = 'E:\\Models\\newshape_mrcnn\\shapes20180621T1554\\mask_rcnn_shapes_0565.h5'

model, dataset_train, dataset_val, train_generator, val_generator, config = \
    prep_newshapes_train2(init_with = model_file, config=config)

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


 windows  Windows
Tensorflow Version: 1.6.0   Keras Version : 2.1.4 
 Initialize config object - super
(56, 56)
 Min Shapes Per Image:  1
 Max Shapes Per Image:  15
 Min Shapes Per Image:  1
 Max Shapes Per Image:  15
>>> Initialize model WITHOUT MASKING LAYERS!!!!
    set_log_dir: Checkpoint path set to : E:\models\newshape_fcn\shapes20180701T2057\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 0 

>>> Resnet Graph 
     Input_image shape : (?, 128, 128, 3)
     After ZeroPadding2D  : (?, 134, 134, 3) (?, 134, 134, 3)
     After Conv2D padding : (?, 64, 64, 64) (?, 64, 64, 64)
     After BatchNorm      : (?, 64, 64, 64) (?, 64, 64, 64)
     C1 Shape: (?, 32, 32, 64) (?, 32, 32, 64)
     C2 Shape:  (?, 32, 32, 256) (?, 32, 32, 256)
     C3 Shape:  (?, 16, 16, 512) (?, 16, 16, 512)
     C4 Shape:  (?, 8, 8, 1024) (?, 8, 8, 1024)
     C5 Shape:  (?, 4, 4, 2048) (?, 4, 4, 2048)

>>> Feature Pyramid Network (FPN) Graph 
     FPN P2 shape : (None, 32, 32, 256)
     FPN P3

    in_tensor_flattened     :  (?, ?)
    Scores shape            :  (448, 3)
    boxes_scores (rehspaed) :  (?, ?, ?, ?)
    gauss_heatmap final shape :  (2, 128, 128, 7)  Keras tensor  False
    gauss_scores  final shape :  (?, ?, ?, ?)  Keras tensor  False
    complete
     pred_cls_cnt shape :  (2, 7) Keras tensor  True
     gt_cls_cnt shape   :  (2, 7) Keras tensor  True
     pred_heatmap_norm  :  (2, 128, 128, 7) Keras tensor  False
     pred_heatmap_scores:  (?, ?, ?, ?) Keras tensor  False
     gt_heatmap_norm    :  (2, 128, 128, 7) Keras tensor  False
     gt_heatmap_scores  :  (?, ?, ?, ?) Keras tensor  False
     complete
<<<  shape of pred_heatmap   :  (2, 128, 128, 7)  Keras tensor  True
<<<  shape of gt_heatmap     :  (2, 128, 128, 7)  Keras tensor  True


---------------------------------------------------
    building Loss Functions 
---------------------------------------------------
 target_class_ids  : True (None, 32)

>>> rpn_bbox_loss_graph
    rpn_match size : (?,

    load_weights: Log directory set to : E:\Models\newshape_mrcnn\shapes20180621T1554\mask_rcnn_shapes_0565.h5
    set_log_dir: Checkpoint path set to : E:\models\newshape_fcn\shapes20180621T1554\mask_rcnn_shapes_{epoch:04d}.h5
    set_log_dir: self.epoch set to 566 
    Load weights complete :  E:\Models\newshape_mrcnn\shapes20180621T1554\mask_rcnn_shapes_0565.h5
Load weights complete E:\Models\newshape_mrcnn\shapes20180621T1554\mask_rcnn_shapes_0565.h5

Configuration Parameters:
-------------------------
BACKBONE_SHAPES                [[32 32]
 [16 16]
 [ 8  8]
 [ 4  4]
 [ 2  2]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
CHECKPOINT_FOLDER              E:\models\newshape_fcn
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
EPOCHS_TO_RUN                  300
FCN_INPUT_SHAPE                [128 128]
FCN_LAYERS                     False
GPU_COU

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 4)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names, limit=6)

###  Print some model information 

In [ ]:
print('\n Inputs: ') 
for i, out in enumerate(model.keras_model.inputs):
    print(i , '    ', out)

print('\n Outputs: ') 
for i, out in enumerate(model.keras_model.outputs):
    print(i , '    ', out)


In [ ]:
print('\n Outputs: ') 
pp.pprint(model.keras_model.outputs)
# print('\n Losses (model.metrics_names): ') 
# pp.pprint(model.get_deduped_metrics_names())
# model.keras_model.summary(line_length = 150) 

## Training - FCN

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

    - #### Or now we can pass a list of layers we want to train in layers !
2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [ ]:
model.config.MIN_LR


In [2]:

train_layers = ['fcn']
loss_names   = ["fcn_norm_loss"]
# config.VALIDATION_STEPS= 125
# config.EPOCHS_TO_RUN          = 100
config.STEPS_PER_EPOCH        = 1
         
model.epoch = 0

model.train(dataset_train, dataset_val, 
            learning_rate = config.LEARNING_RATE, 
            epochs_to_run = config.EPOCHS_TO_RUN,
#             epochs = 25,            # total number of epochs to run (accross multiple trainings)
            layers = train_layers,
            losses = loss_names)

['fcn']
['(fcn\\_.*)']
layers regex : (fcn\_.*)
 213  fcn_block1_conv1       (Conv2D              )   TRAIN 
 214  fcn_block1_conv2       (Conv2D              )   TRAIN 
 216  fcn_block2_conv1       (Conv2D              )   TRAIN 
 217  fcn_block2_conv2       (Conv2D              )   TRAIN 
 219  fcn_block3_conv1       (Conv2D              )   TRAIN 
 220  fcn_block3_conv2       (Conv2D              )   TRAIN 
 221  fcn_block3_conv3       (Conv2D              )   TRAIN 
 223  fcn_block4_conv1       (Conv2D              )   TRAIN 
 224  fcn_block4_conv2       (Conv2D              )   TRAIN 
 225  fcn_block4_conv3       (Conv2D              )   TRAIN 
 227  fcn_block5_conv1       (Conv2D              )   TRAIN 
 228  fcn_block5_conv2       (Conv2D              )   TRAIN 
 229  fcn_block5_conv3       (Conv2D              )   TRAIN 
 231  fcn_fc1                (Conv2D              )   TRAIN 
 233  fcn_fc2                (Conv2D              )   TRAIN 
 235  fcn_classify           (Conv2D 

    params:  <tf.Variable 'fcn_block5_conv3/kernel:0' shape=(3, 3, 512, 512) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_24/Merge:0", shape=(3, 3, 512, 512), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable_24:0' shape=(3, 3, 512, 512) dtype=float32_ref>
    params:  <tf.Variable 'fcn_block5_conv3/bias:0' shape=(512,) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_25/Merge:0", shape=(512,), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable_25:0' shape=(512,) dtype=float32_ref>
    params:  <tf.Variable 'fcn_fc1/kernel:0' shape=(7, 7, 512, 2048) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_26/Merge:0", shape=(7, 7, 512, 2048), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable_26:0' shape=(7, 7, 512, 2048) dtype=float32_ref>
    params:  <tf.Variable 'fcn_fc1/bias:0' shape=(2048,) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_27/Merge:0", shape=(2048,), dtype=float32)
  moements:  <tf.Variable 't

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\keras\utils\generic_utils.py:408: RuntimeWarning: invalid value encountered in greater
  if abs(avg) > 1e-3:
D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\keras\callbacks.py:438: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):
D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\keras\callbacks.py:940: RuntimeWarning: invalid value encountered in less
  self.monitor_op = lambda a, b: np.less(a, b - self.epsilon)
D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\keras\callbacks.py:530: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):


callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  fcn_norm_loss   v: [[0.0195]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  loss   v: [[nan]]
 generic_utils.(332) values:  loss  - [[nan]]
 generic_utils.(332) values:  fcn_norm_loss  - [[0.0195]]
 generic_utils.(332) values:  loss  - [[nan]]
 generic_utils.(332) values:  fcn_norm_loss  - [[0.0195]]
 generic_utils.(332) values:  val_loss  - [[nan]]
 generic_utils.(332) values:  val_fcn_norm_loss  - [[0.0213]]
1/1 [==============================] - 2s 2s/step - loss: nan - fcn_norm_loss: 0.0195 - val_loss: nan - val_fcn_norm_loss: 0.0213

Epoch 00002: val_loss did not improve
Epoch 3/300
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  fcn_norm_loss   v: [[0.02]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229

 generic_utils.(332) values:  loss  - [[nan]]
 generic_utils.(332) values:  fcn_norm_loss  - [[0.0191]]
 generic_utils.(332) values:  loss  - [[nan]]
 generic_utils.(332) values:  fcn_norm_loss  - [[0.0191]]
 generic_utils.(332) values:  val_loss  - [[nan]]
 generic_utils.(332) values:  val_fcn_norm_loss  - [[0.0213]]
1/1 [==============================] - 2s 2s/step - loss: nan - fcn_norm_loss: 0.0191 - val_loss: nan - val_fcn_norm_loss: 0.0213

Epoch 00012: val_loss did not improve
Epoch 13/300
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  fcn_norm_loss   v: [[0.0121]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  loss   v: [[nan]]
 generic_utils.(332) values:  loss  - [[nan]]
 generic_utils.(332) values:  fcn_norm_loss  - [[0.0121]]
 generic_utils.(332) values:  loss  - [[nan]]
 generic_utils.(332) values:  fcn_norm_loss  - [[0.

## Training mrcnn, fpn, rpn layers

In [3]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
# Wed 09-05-2018
# train_layers = ['mrcnn', 'fpn','rpn']
# loss_names   = [  "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss", "mrcnn_mask_loss"]
train_layers = ['mrcnn', 'fpn','rpn']
loss_names   = [  "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss" ]
config.STEPS_PER_EPOCH        = 8

model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs_to_run = config.EPOCHS_TO_RUN,
#             epochs = 2500,
#             epochs_to_run =2, 
            layers = train_layers,
            losses= loss_names
            )

['mrcnn', 'fpn', 'rpn']
['(mrcnn\\_.*)', '(fpn\\_.*)', '(rpn\\_.*)']
layers regex : (mrcnn\_.*)|(fpn\_.*)|(rpn\_.*)
 174  fpn_c5p5               (Conv2D              )   TRAIN 
 176  fpn_c4p4               (Conv2D              )   TRAIN 
 179  fpn_c3p3               (Conv2D              )   TRAIN 
 182  fpn_c2p2               (Conv2D              )   TRAIN 
 184  fpn_p5                 (Conv2D              )   TRAIN 
 185  fpn_p2                 (Conv2D              )   TRAIN 
 186  fpn_p3                 (Conv2D              )   TRAIN 
 187  fpn_p4                 (Conv2D              )   TRAIN 
       1  rpn_conv_shared        (Conv2D              )   TRAIN 
       2  rpn_class_raw          (Conv2D              )   TRAIN 
       4  rpn_bbox_pred          (Conv2D              )   TRAIN 
 198  mrcnn_class_conv1      (TimeDistributed     )   TRAIN 
 199  mrcnn_class_bn1        (TimeDistributed     )   TRAIN 
 201  mrcnn_class_conv2      (TimeDistributed     )   TRAIN 
 202  mrcnn_class_

D:\Program Files\Anaconda3\envs\TF_gpu\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


    params:  <tf.Variable 'fpn_c5p5/kernel:0' shape=(1, 1, 2048, 256) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond/Merge:0", shape=(1, 1, 2048, 256), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable:0' shape=(1, 1, 2048, 256) dtype=float32_ref>
    params:  <tf.Variable 'fpn_c5p5/bias:0' shape=(256,) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_1/Merge:0", shape=(256,), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable_1:0' shape=(256,) dtype=float32_ref>
    params:  <tf.Variable 'fpn_c4p4/kernel:0' shape=(1, 1, 1024, 256) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_2/Merge:0", shape=(1, 1, 1024, 256), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable_2:0' shape=(1, 1, 1024, 256) dtype=float32_ref>
    params:  <tf.Variable 'fpn_c4p4/bias:0' shape=(256,) dtype=float32_ref>
 gradients:  Tensor("training/SGD/cond_3/Merge:0", shape=(256,), dtype=float32)
  moements:  <tf.Variable 'training/SGD/Variable_3:0

Epoch 567/866
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  mrcnn_class_loss   v: [[0.1457]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  rpn_class_loss   v: [[0.0401]]
    on_batch_end (229):  logs.items() k:  mrcnn_bbox_loss   v: [[0.1307]]
    on_batch_end (229):  logs.items() k:  loss   v: [[0.5611]]
    on_batch_end (229):  logs.items() k:  rpn_bbox_loss   v: [[0.2447]]
 generic_utils.(332) values:  loss  - [[0.5611]]
 generic_utils.(332) values:  rpn_class_loss  - [[0.0401]]
 generic_utils.(332) values:  rpn_bbox_loss  - [[0.2447]]
 generic_utils.(332) values:  mrcnn_class_loss  - [[0.1457]]
 generic_utils.(332) values:  mrcnn_bbox_loss  - [[0.1307]]
1/8 [==>...........................] - ETA: 28s - loss: 0.5611 - rpn_class_loss: 0.0401 - rpn_bbox_loss: 0.2447 - mrcnn_class_loss: 0.1457 - mrcnn_bbox_loss: 0.1307callbacks (22


Epoch 00567: val_loss improved from inf to 0.17444825, saving model to E:\models\newshape_fcn\shapes20180621T1554\mask_rcnn_shapes_0567.h5
Epoch 568/866
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  mrcnn_class_loss   v: [[0.1759]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  rpn_class_loss   v: [[0.0195]]
    on_batch_end (229):  logs.items() k:  mrcnn_bbox_loss   v: [[0.067]]
    on_batch_end (229):  logs.items() k:  loss   v: [[0.5092]]
    on_batch_end (229):  logs.items() k:  rpn_bbox_loss   v: [[0.2468]]
 generic_utils.(332) values:  loss  - [[0.5092]]
 generic_utils.(332) values:  rpn_class_loss  - [[0.0195]]
 generic_utils.(332) values:  rpn_bbox_loss  - [[0.2468]]
 generic_utils.(332) values:  mrcnn_class_loss  - [[0.1759]]
 generic_utils.(332) values:  mrcnn_bbox_loss  - [[0.067]]
1/8 [==>...........................] - 

...


Epoch 00587: val_loss did not improve
Epoch 588/866
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  mrcnn_class_loss   v: [[0.0627]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  rpn_class_loss   v: [[0.0209]]
    on_batch_end (229):  logs.items() k:  mrcnn_bbox_loss   v: [[0.0831]]
    on_batch_end (229):  logs.items() k:  loss   v: [[0.2673]]
    on_batch_end (229):  logs.items() k:  rpn_bbox_loss   v: [[0.1007]]
 generic_utils.(332) values:  loss  - [[0.2673]]
 generic_utils.(332) values:  rpn_class_loss  - [[0.0209]]
 generic_utils.(332) values:  rpn_bbox_loss  - [[0.1007]]
 generic_utils.(332) values:  mrcnn_class_loss  - [[0.0627]]
 generic_utils.(332) values:  mrcnn_bbox_loss  - [[0.0831]]
1/8 [==>...........................] - ETA: 2s - loss: 0.2673 - rpn_class_loss: 0.0209 - rpn_bbox_loss: 0.1007 - mrcnn_class_loss: 0.0627 


Epoch 00588: val_loss did not improve
Epoch 589/866
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  mrcnn_class_loss   v: [[0.0501]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  rpn_class_loss   v: [[0.0179]]
    on_batch_end (229):  logs.items() k:  mrcnn_bbox_loss   v: [[0.0694]]
    on_batch_end (229):  logs.items() k:  loss   v: [[0.3615]]
    on_batch_end (229):  logs.items() k:  rpn_bbox_loss   v: [[0.2241]]
 generic_utils.(332) values:  loss  - [[0.3615]]
 generic_utils.(332) values:  rpn_class_loss  - [[0.0179]]
 generic_utils.(332) values:  rpn_bbox_loss  - [[0.2241]]
 generic_utils.(332) values:  mrcnn_class_loss  - [[0.0501]]
 generic_utils.(332) values:  mrcnn_bbox_loss  - [[0.0694]]
1/8 [==>...........................] - ETA: 2s - loss: 0.3615 - rpn_class_loss: 0.0179 - rpn_bbox_loss: 0.2241 - mrcnn_class_loss: 0.0501 


Epoch 00589: val_loss did not improve
Epoch 590/866
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  mrcnn_class_loss   v: [[0.084]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  rpn_class_loss   v: [[0.039]]
    on_batch_end (229):  logs.items() k:  mrcnn_bbox_loss   v: [[0.1372]]
    on_batch_end (229):  logs.items() k:  loss   v: [[0.5661]]
    on_batch_end (229):  logs.items() k:  rpn_bbox_loss   v: [[0.3059]]
 generic_utils.(332) values:  loss  - [[0.5661]]
 generic_utils.(332) values:  rpn_class_loss  - [[0.039]]
 generic_utils.(332) values:  rpn_bbox_loss  - [[0.3059]]
 generic_utils.(332) values:  mrcnn_class_loss  - [[0.084]]
 generic_utils.(332) values:  mrcnn_bbox_loss  - [[0.1372]]
1/8 [==>...........................] - ETA: 3s - loss: 0.5661 - rpn_class_loss: 0.0390 - rpn_bbox_loss: 0.3059 - mrcnn_class_loss: 0.0840 - mr


Epoch 00590: val_loss did not improve
Epoch 591/866
callbacks (228): Baselogger: on_batch_end() callback  2
    on_batch_end (229):  logs.items() k:  mrcnn_class_loss   v: [[0.0338]]
    on_batch_end (229):  logs.items() k:  batch   v: 0
    on_batch_end (229):  logs.items() k:  size   v: 2
    on_batch_end (229):  logs.items() k:  rpn_class_loss   v: [[0.0141]]
    on_batch_end (229):  logs.items() k:  mrcnn_bbox_loss   v: [[0.0315]]
    on_batch_end (229):  logs.items() k:  loss   v: [[0.1585]]
    on_batch_end (229):  logs.items() k:  rpn_bbox_loss   v: [[0.0791]]
 generic_utils.(332) values:  loss  - [[0.1585]]
 generic_utils.(332) values:  rpn_class_loss  - [[0.0141]]
 generic_utils.(332) values:  rpn_bbox_loss  - [[0.0791]]
 generic_utils.(332) values:  mrcnn_class_loss  - [[0.0338]]
 generic_utils.(332) values:  mrcnn_bbox_loss  - [[0.0315]]
1/8 [==>...........................] - ETA: 3s - loss: 0.1585 - rpn_class_loss: 0.0141 - rpn_bbox_loss: 0.0791 - mrcnn_class_loss: 0.0338 

KeyboardInterrupt: 

## - Training heads using train_in_batches ()

We need to use this method for the time being as the fit generator does not have provide EASY access to the output in Keras call backs. By training in batches, we pass a batch through the network, pick up the generated RoI detections and bounding boxes and generate our semantic / gaussian tensors ...


In [ ]:
model.train_in_batches(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/6, 
            epochs_to_run = 3,
            layers='heads')

## Fine Tuning
Fine tune all layers

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=211,
            layers="all")

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes_2500.h5")
model.keras_model.save_weights(model_path)

###  Define Data Generators, get next shapes from generator and display loaded shapes

### Define Data Generator

In [ ]:
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=model.config.BATCH_SIZE,
                                 augment = False)
val_generator = data_generator(dataset_val, model.config, shuffle=True, 
                                batch_size=model.config.BATCH_SIZE,
                                augment=False)

### Get next shapes from generator and display loaded shapes

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)



### Push Data thru model using get_layer_output()

In [ ]:

layers_out = get_layer_output_2(model.keras_model, train_batch_x, 1)


In [ ]:
input_gt_class_ids = train_batch_x[4]

target_class_ids = layers_out[5]
mrcnn_class_logits = layers_out[9]
rpn_class_loss   = layers_out[13]
rpn_bbox_loss    = layers_out[14]
mrcnn_class_loss = layers_out[15]
mrcnn_bbox_loss  = layers_out[16]
mrcnn_mask_loss  = layers_out[17]
active_class_ids = layers_out[20]
# pred_masks = tf.identity(layers_out[18])
# gt_masks   = tf.identity(layers_out[19])

# shape = KB.int_shape(pred_masks)

In [ ]:
print(rpn_class_loss, rpn_bbox_loss)
print(mrcnn_class_loss, mrcnn_bbox_loss, mrcnn_mask_loss)
print(active_class_ids)
print()
print(target_class_ids[1])
print()
print(mrcnn_class_logits[1])
print('gt class ids')
print(input_gt_class_ids)

###  Simulate `mrcnn_class_loss` computation

In [ ]:
print('\n>>> mrcnn_class_loss_graph ' )
print('    target_class_ids  size :', target_class_ids.shape)
print('    pred_class_logits size :', mrcnnpred_class_logits.shape)
print('    active_class_ids  size :', active_class_ids.shape)    
target_class_ids = tf.cast(target_class_ids, 'int64')

# Find predictions of classes that are not in the dataset.

pred_class_ids = tf.argmax(pred_class_logits, axis=2)

# TODO: Update this line to work with batch > 1. Right now it assumes all
#       images in a batch have the same active_class_ids
pred_active = tf.gather(active_class_ids[0], pred_class_ids)

# Loss
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=target_class_ids, logits=pred_class_logits)

# Erase losses of predictions of classes that are not in the active
# classes of the image.
loss = loss * pred_active

# Computer loss mean. Use only predictions that contribute
# to the loss to get a correct mean.
loss = tf.reduce_sum(loss) / tf.reduce_sum(pred_active)
loss = KB.reshape(loss, [1, 1])
return loss


## Plot Predicted and Ground Truth Probability Heatmaps `pred_gaussian` and `gt_gaussian` (Tensorflow)

`pred_gaussian2` and `gt_gaussian2` from Tensorflow PCN layer

In [ ]:
# gt_heatmap  = layers_out[27]     # gt_gaussiam 
# pred_heatmap= layers_out[24]  # pred_gaussian
gt_heatmap  = layers_out[21]     # gt_gaussiam 
pred_heatmap= layers_out[18]  # pred_gaussian
print('gt_gaussian heatmap shape : ', gt_heatmap.shape, ' pred_gaussian heatmap shape: ', pred_heatmap.shape)
num_images = 1 # config.IMAGES_PER_GPU
num_classes = config.NUM_CLASSES

img = 2

image_id = img_meta[img,0]
print('Image id: ',image_id)
print('Classes (1: circle, 2: square, 3: triangle ): ')
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


for cls in range(num_classes):
    ttl = 'GROUND TRUTH HEATMAP - image :  {} class: {} '.format(img,cls)
    print(' *** Zout  ', gt_heatmap[img,:,:,cls].shape, ttl)   
    plot_gaussian( gt_heatmap[img,:,:,cls], title = ttl)
    
    ttl = 'PREDICTED heatmap  - image :  {} class: {} '.format(img,cls)     
    print(' *** pred_heatmap ', pred_heatmap[img,:,:,cls].shape, ttl)   
    plot_gaussian(pred_heatmap[img,:,:,cls], title = ttl)  



### Plot Output from FCN network `fcn_bilinear` and compare with `pred_gaussian`

In [ ]:
from mrcnn.visualize import plot_gaussian
import matplotlib as plt

%matplotlib inline
img = 2
image_id = img_meta[img,0]
print('Image id: ',image_id)
print('Classes (1: circle, 2: square, 3: triangle ): ')
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)


Zout  = layers_out[21]     # gt_gaussiam 
Zout2 = layers_out[12]     # fcn_bilinear

print(Zout.shape, Zout2.shape)
num_images = config.IMAGES_PER_GPU
num_classes = config.NUM_CLASSES


for cls in range(num_classes):
    ttl = 'GroundTruth - image :  {} class: {} '.format(img,cls)
    print(' *** Zout  ', Zout[img,:,:,cls].shape, ttl)   
    plot_gaussian( Zout[img,:,:,cls], title = ttl)
    
    ttl = 'FCN_Bilinear- image :  {} class: {} '.format(img,cls)     
    print(' *** Zout2 ', Zout2[img,:,:,cls].shape, ttl)   
    plot_gaussian(Zout2[img,:,:,cls], title = ttl)  


### Display ground truth bboxes from Shapes database (using `load_image_gt` )

Here we are displaying the ground truth bounding boxes as provided by the dataset

In [ ]:
img = 0
image_id = img_meta[img,0]
print('Image id: ',image_id)
p_original_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
# print(p_gt_class_id.shape, p_gt_bbox.shape, p_gt_mask.shape)
print(p_gt_bbox[0:3,:])
print(p_gt_class_id)
visualize.draw_boxes(p_original_image, p_gt_bbox[0:3])

# image_id = img_meta[img,0]
# print('Image id: ',image_id)
# p_original_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
#             load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
# # print(p_gt_class_id.shape, p_gt_bbox.shape, p_gt_mask.shape)
# print(p_gt_bbox)
# print(p_gt_class_id)
# visualize.draw_boxes(p_original_image, p_gt_bbox)

### Display Predicted  Ground Truth Bounding Boxes  `gt_tensor` and `gt_tensor2`

layers_out[22]  `gt_tensor` is based on input_gt_class_ids and input_normlzd_gt_boxes
layers_out[28]  `gt_tensor2` is based on input_gt_class_ids and input_normlzd_gt_boxes, generated using Tensorflow

Display the Ground Truth bounding boxes from the tensor we've constructed

In [ ]:
from mrcnn.utils  import stack_tensors, stack_tensors_3d
# print(gt_bboxes)
# visualize.display_instances(p_original_image, p_gt_bbox, p_gt_mask, p_gt_class_id, 
#                             dataset_train.class_names, figsize=(8, 8))
# pp.pprint(gt_bboxes)
img = 0
image_id = img_meta[img,0]

print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)   
gt_bboxes_stacked = stack_tensors_3d(layers_out[22][img])
print(gt_bboxes_stacked)
visualize.draw_boxes(p_image, gt_bboxes_stacked[0:2,2:6])


## Display RoI proposals `pred_bboxes` generated for one class

Display bounding boxes from tensor of proposals produced by the network 
Square: 1 , Circle:2 , Triangle 3

In [ ]:
img = 0
cls = 1 # <==== Class to display
pred_tensor = layers_out[19]   # numpy pred_tesnor
# pred_tensor = layers_out[25]   # tensorflow pred_tensor 

image_id = img_meta[img,0]
print('Image id: ',image_id)
p_image, p_image_meta, p_gt_class_id, p_gt_bbox, p_gt_mask =  \
            load_image_gt(dataset_train, config, image_id, augment=False, use_mini_mask=True)
print(p_image_meta)
print(pred_tensor[img,cls,:].shape)
print(pred_tensor[img,cls])
#+'-'+str(np.around(int(x[1]),decimals = 3))
# class id: str(int(x[6]))+'-'+
caps = [str(int(x[0]))+'-'+str(np.around(x[1],decimals = 3))  for x in pred_tensor[img,cls,:].tolist() ]
print(caps)

visualize.draw_boxes(p_image, pred_tensor[img,cls,:,2:6], captions = caps)

In [ ]:
layers_out[0][0] * [128, 128,128,128]   #output_rois*

### Calculate  mrcnn_bbox_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss

target_class_ids = layers_out[1][0:1]
target_bbox      = layers_out[2][0:1]
mrcnn_bbox       = layers_out[10][0:1]
mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids

print('target_class_ids', target_class_ids.shape)
print(target_class_ids)  # tgt_class_ids
print(' class with max probability', mrcnn_class_ids.shape)
print(mrcnn_class_ids)
print('target_bboxes', target_bbox.shape)
# print(target_bbox)  # tgt_bounding boxes
print('mrcnn_bboxes',mrcnn_bbox.shape)
# print(mrcnn_bbox)  #mrcnn_bboxes
pred_bbox = mrcnn_bbox

# calc mrcnn_bbox_loss
target_class_ids = K.reshape(target_class_ids, (-1,))
print(target_class_ids.shape)
target_bbox      = K.reshape(target_bbox, (-1, 4))
print('target_bboxx: ', target_bbox.shape)
pred_bbox        = K.reshape(pred_bbox, (-1, pred_bbox.shape[2], 4))
print('pred_bbox : ', pred_bbox.shape)

positive_roi_ix        = tf.where(target_class_ids > 0)[:, 0]
print(positive_roi_ix.eval())
positive_roi_class_ids = tf.cast( tf.gather(target_class_ids, positive_roi_ix), tf.int64)
print(positive_roi_class_ids.eval())
indices                = tf.stack([positive_roi_ix, positive_roi_class_ids], axis=1)
print(indices.eval())


target_bbox = tf.gather(target_bbox, positive_roi_ix)
print(target_bbox.eval())
pred_bbox   = tf.gather_nd(pred_bbox, indices)
print(pred_bbox.eval())

print('tf.size ',tf.size(target_bbox).eval())

diff = K.abs(target_bbox - pred_bbox)
print(diff.eval())

less_than_one = K.cast(K.less(diff, 1.0), "float32")
# print(less_than_one.eval())

loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
# print( (1-less_than_one).eval())



# loss        = K.switch(tf.size(target_bbox) > 0,
#                 smooth_l1_loss(y_true=target_bbox, y_pred=pred_bbox),
#                 tf.constant(0.0))
print(loss.eval())
sumloss = K.sum(loss)
print(sumloss.eval())
print((sumloss/40).eval())
meanloss        = K.mean(loss)
print(meanloss.eval())

###  Calculate mrcnn_class_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss

target_class_ids = layers_out[1][0:1]
pred_class_logits = layers_out[8][0:1]
active_class_ids    = np.array([1,1,1,1])

# mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids

print(' target_class_ids', target_class_ids.shape)
print(target_class_ids)  # tgt_class_ids
print(' class logits', pred_class_logits.shape)
print(pred_class_logits)
print(' active, class_ids ', active_class_ids.shape)
print(active_class_ids)  # tgt_bounding boxes

pred_class_ids = tf.argmax(pred_class_logits, axis=2)
print(pred_class_ids.eval())  #mrcnn_bboxes
mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids
print(mrcnn_class_ids)
# pred_bbox = mrcnn_bbox
pred_active = tf.to_float(tf.gather(active_class_ids, pred_class_ids))
print(pred_active.eval())
# calc mrcnn_bbox_loss
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(
       labels=target_class_ids, logits=pred_class_logits)
print(loss.eval())

loss = loss * tf.to_float(pred_active)
print(loss.eval())

print(tf.reduce_sum(loss).eval())
print(tf.reduce_sum(pred_active).eval())
loss = tf.reduce_sum(loss) / tf.reduce_sum(pred_active)
print(loss.eval())

###  Calculate mrcnn_mask_loss

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss

target_class_ids    = layers_out[1][0:3]
target_masks        = layers_out[3][0:3]
pred_masks          = layers_out[11][0:3]
# mrcnn_class_ids  = np.argmax(layers_out[9][0:1],axis = -1)     # mrcnn_class_ids
print('    target_class_ids shape :', target_class_ids.shape)
print('    target_masks     shape :', target_masks.shape)
print('    pred_masks       shape :', pred_masks.shape)    


target_class_ids = K.reshape(target_class_ids, (-1,))
print('    target_class_ids shape :', target_class_ids.shape, '\n', target_class_ids.eval())

mask_shape       = tf.shape(target_masks)
print('    mask_shape       shape :', mask_shape.shape, mask_shape.eval())    

target_masks     = K.reshape(target_masks, (-1, mask_shape[2], mask_shape[3]))
print('    target_masks     shape :', tf.shape(target_masks).eval())        

pred_shape       = tf.shape(pred_masks)
print('    pred_shape       shape :', pred_shape.shape, pred_shape.eval())        

pred_masks       = K.reshape(pred_masks, (-1, pred_shape[2], pred_shape[3], pred_shape[4]))
print('    pred_masks       shape :', tf.shape(pred_masks).eval())        


pred_masks = tf.transpose(pred_masks, [0, 3, 1, 2])
print('    pred_masks       shape :', tf.shape(pred_masks).eval())        

# Only positive ROIs contribute to the loss. And only
# the class specific mask of each ROI.
positive_ix        = tf.where(target_class_ids > 0)[:, 0]
positive_class_ids = tf.cast(tf.gather(target_class_ids, positive_ix), tf.int64)
indices            = tf.stack([positive_ix, positive_class_ids], axis=1)
print(indices.eval())



y_true = tf.gather(target_masks, positive_ix)
print('     y_true shape:', tf.shape(y_true).eval())
y_pred = tf.gather_nd(pred_masks, indices)
print('     y_pred shape:', tf.shape(y_pred).eval())

loss = K.switch(tf.size(y_true) > 0,
                K.binary_crossentropy(target=y_true, output=y_pred),
                tf.constant(0.0))
print(tf.shape(loss).eval())

loss = K.mean(loss)
print('     final loss shape:', tf.shape(loss).eval())
print(loss.eval())
loss = K.reshape(loss, [1, 1])
print('     final loss shape:', tf.shape(loss).eval())
print(loss.eval())

### Calculate a pixel loss on fcn_gaussian and gt_gaussian 

In [ ]:
import keras.backend as K

from mrcnn.utils import apply_box_deltas
from mrcnn.loss  import smooth_l1_loss
pred_masks          = layers_out[12][0:3]
target_masks        = layers_out[27][0:3]

print('    target_masks     shape :', tf.shape(target_masks).eval())
print('    pred_masks       shape :', tf.shape(pred_masks).eval())    

diff = K.abs(target_masks - pred_masks)
print(tf.shape(diff).eval())

less_than_one = K.cast(K.less(diff, 1.0), "float32")
print(tf.shape(less_than_one).eval())

loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
print(tf.shape(loss).eval())

# print( (1-less_than_one).eval())

# loss = K.switch(tf.size(y_true) > 0,
#                 K.binary_crossentropy(target=y_true, output=y_pred),
#                 tf.constant(0.0))
meanloss = K.mean(loss)
print(tf.shape(meanloss).eval())
print(meanloss.eval())
# loss = K.reshape(loss, [1, 1])
# print('     final loss shape:', loss.get_shape())
# return loss


mask_shape       = tf.shape(target_masks)
print('    mask_shape       shape :', tf.shape(mask_shape).eval())    

target_masks     = K.reshape(target_masks, (-1, mask_shape[1], mask_shape[2]))
print('    target_masks     shape :', tf.shape(target_masks).eval())        

pred_shape       = tf.shape(pred_masks)
print('    pred_shape       shape :', tf.shape(pred_shape).eval())        

pred_masks       = K.reshape(pred_masks, (-1, pred_shape[1], pred_shape[2]))
print('    pred_masks       shape :', tf.shape(pred_masks).eval())
# Permute predicted masks to [N, num_classes, height, width]
# diff = K.abs(target_masks - pred_masks)
# print(tf.shape(diff).eval())

# less_than_one = K.cast(K.less(diff, 1.0), "float32")
# print(tf.shape(less_than_one).eval())

# loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)
# print(tf.shape(loss).eval())

# meanloss = K.mean(loss)
# print(tf.shape(meanloss).eval())
# print(meanloss.eval())

loss = K.switch(tf.size(target_masks) > 0,
                smooth_l1_loss(y_true=target_masks, y_pred=pred_masks),
                tf.constant(0.0))
loss = K.mean(loss)
loss = K.reshape(loss, [1, 1])
print('     final loss shape:', loss.get_shape())
print(loss.eval())

###  Mean values of GT, Pred, and FCN heatmaps 

In [ ]:
pred_masks = tf.identity(layers_out[24])
gt_masks = tf.identity(layers_out[27])
fcn_masks = tf.identity(layers_out[12])
print(gt_masks.shape, fcn_masks.shape)
for img in range(5):
    for cls in range(4):
        gt_mean = K.mean(gt_masks[img,:,:,cls])
        fcn_mean= K.mean(fcn_masks[img,:,:,cls])
        pred_mean= K.mean(pred_masks[img,:,:,cls])
        print('Img/Cls: ', img, '/', cls,'    gtmean: ', gt_mean.eval(), '\t fcn : ' , fcn_mean.eval(), '\t pred :', pred_mean.eval())

In [ ]:
img  = 0
class_probs = layers_out[9][img]   # mrcnn_class
deltas      = layers_out[10][img]       # mrcnn_bbox

print(class_probs.shape)
print('class probabilities')
print(class_probs)
class_ids = np.argmax(layers_out[9][img],axis = 1)     # mrcnn_class_ids
print(' class with max probability')
print(class_ids)


# layers_out[10][2,0,3]
print('deltas.shape :', deltas.shape)
print(deltas[0:4])

deltas_specific = deltas[np.arange(32),class_ids]
print('deltas of max prob class: ', deltas_specific.shape)
print(deltas_specific[0:5])
output_rois = layers_out[0][img]*[128,128,128,128]
print('output_rois: ', output_rois.shape)
print(output_rois[0:])

refined_rois    = apply_box_deltas(output_rois, deltas_specific * config.BBOX_STD_DEV)
print('refined rois: ',refined_rois.shape)
print(refined_rois)

In [ ]:
img = 0
cls = 0
fcn_out = layers_out[12][img]
fcn_sum = np.sum(fcn_out, axis=(0,1))
print(fcn_sum)
for cls in range(4):
    print('min :', np.min(fcn_out[:,:,cls]), 'max :', np.max(fcn_out[:,:,cls]), )

In [ ]:
print(train_batch_x[4][2])
print(train_batch_x[5][2]/[128,128,128,128])